<img src="../assets/data_analysis_with_polars_copyright-1.png" width="600"/>

This notebook is a free sample from my Data Analysis with Polars course on Udemy.

Use this link to do the full course at half price: https://www.udemy.com/course/data-analysis-with-polars/?couponCode=POLARS_HALF_PRICE2


# Filtering rows 2: Using `filter` and the Expression API

By the end of this lecture you will be able to:
- select rows with the `filter` method
- add a row number column
- select rows with multiple conditions
- explain the difference between `[]` and `filter`

The `filter` method is our first example of the *Expression API*.

_**Learning to use the *Expression API* is the most important step to writing high performance queries in Polars**_


In [2]:
import polars as pl

In [3]:
csvFile = "../data/titanic.csv"

In [4]:
df = pl.read_csv(csvFile)
df.head(3)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow...","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. ...","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Mis...","""female""",26.0,0,0,"""STON/O2. 31012...",7.925,null,"""S"""


## Selecting rows with `filter`

We first use an *expression* in the `filter` method before we examine the syntax in more detail.

In this example we choose all the first class passengers.

In [ ]:
df.filter(
    pl.col('Pclass') == 1
).head(3)

## Syntax of `filter`
Inside the `filter` method we pass our first _**expression**_ and apply a Boolean condition to it:

`pl.col('Pclass') == 1`

This expression has two parts:
- `pl.col('Pclass')` expression selects the `Pclass` column from `df`
- `== 1` applies a Boolean condition to this expression

In this example we choose all rows with the number of parents & children (`Parch`) is greater than 1

In [ ]:
df.filter(
    pl.col('Parch') > 1
).head(3)

## Key differences between `[]` and `filter`

- `[]` indexing can only be used in eager mode, `filter` can also be used in lazy mode
- `filter` expressions can be **optimised** in lazy mode by the query optimiser

Use cases for `[]`:
- inspecting data in interactive mode

Use cases for `filter`:
- all other times

### Conditions based on row numbers

We can add an explicit row number column using `with_row_count` on a `DataFrame`

In [ ]:
df = pl.read_csv(csvFile)
df = df.with_row_count(name='row_nr')
df.head(3)

We can then use `filter` to apply a condition based on row number

In [ ]:
df.filter(
    pl.col('row_nr') < 4
)

## Select rows with multiple conditions

### Apply `AND` conditions

We can apply filter `AND` conditions by **chaining** calls to `filter`.

In [ ]:
dfFiltered = (
    pl.read_csv(csvFile)
    .filter(
        pl.col('Pclass') == 1
    )
    .filter(
        pl.col('Age') > 70
    )
)
dfFiltered.head(3)

In eager mode chaining requires scanning the `DataFrame` for each `filter` call.

It is better to **concatenate** multiple `AND` conditions in a single `filter` call using `&`

In [ ]:
df.filter(
    (pl.col('Age') > 70) & (pl.col('Pclass') == 1)
).head(3)

### Apply `AND` condition on a range

We use `in_between` to apply a condition on a range. In this case we are looking for values greater than 10 and less than 13

In [ ]:
(
    df
    .filter(
        pl.col("Age").is_between(10,13)
    )
)

We use the `closed` argument to specify if we want the range to be open, closed on both sides or open on the left or right. The default is for the range to be open (with a value of `"none"`). 

In this example we are looking for values from 10 to 13 inclusive of the boundaries

In [ ]:
(
    df
    .filter(
        pl.col("Age").is_between(10,13,closed="both")
    )
)

### Apply `OR` conditions

We can apply an OR filter using the pipe `|` operator

In [ ]:
df.filter(
    (pl.col('Age') > 70) | (pl.col('Pclass') == 1)
).head(3)

If our `OR` statement is only testing for equality on a single column we can use `is_in`

In [7]:
df.filter(
    pl.col('Pclass').is_in([2,3])
).head(3)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow...","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
3,1,3,"""Heikkinen, Mis...","""female""",26.0,0,0,"""STON/O2. 31012...",7.925,null,"""S"""
5,0,3,"""Allen, Mr. Wil...","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


# Exercises
In the exercises you will develop your understanding of
- using the `filter` method
- adding a row number column
- using the `filter` method with `AND` and `OR` conditions

### Exercise 1 
Select all rows where `Age` is greater than 30

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
)

### Exercise 2 

In this exercise we filter on row numbers.

First add a row number column

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
)

Continue by selecting the first 5 rows using `filter` on the row number column

### Exercise 3

Select all rows where `Age` is greater than 30 and the passenger was in 2nd class

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(<blank>)
    .head(3)
)

### Exercise 4 
Select all rows where `Age` is less than or equal 5 *or* `Age` is greater than 75

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(<blank>)
    .head(3)
)

Select all rows where `Age` is in between 5 and 75 including the upper boundary 

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(<blank>)
    .head(3)
)

## Solutions

### Solution to Exercise 1
Select all rows with `Age` greater than 30

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(pl.col('Age') > 30)
    .head(3)
)

### Solution to Exercise 2
Add a row number column

In [ ]:
(
    pl.read_csv(csvFile)
    .with_row_count("row_nr")
)

Continue by selecting the first 5 rows using `filter` on the row number column

In [ ]:
(
    pl.read_csv(csvFile)
    .with_row_count("row_nr")
    .filter(pl.col("row_nr")<5)
)

### Solution to Exercise 3
Select all rows where `Age` is greater than 30 and the passenger was in 2nd class

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(
        (pl.col('Age') > 30) & (pl.col('Pclass')==2)
    )
    .head(3)
)

### Solution to Exercise 4
Select all rows where `Age` is less than or equal 5 *or* `Age` is greater than 75

In [ ]:
(
    pl.read_csv(csvFile)
    .filter( (pl.col("Age") <= 5) | (pl.col("Age") > 75) )
    .head(3)
)

In [ ]:
(
    pl.read_csv(csvFile)
    .filter( pl.col("Age").is_between(5,75,closed="right") )
    .head(3)
)